In [1]:
# Based on code from https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

import nltk
import spacy
import gensim
import random
import pickle
import pandas as pd
from gensim import corpora
from spacy.lang.en import English
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

spacy.load('en_core_web_sm')
#spacy.load('es_core_news_sm')
#spacy.load("fr_core_news_sm")
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))
#en_stop = set(nltk.corpus.stopwords.words('spanish'))
#en_stop = set(nltk.corpus.stopwords.words('french'))

parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to /Users/fv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/fv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def get_topics(path, num_topics):
    text_data = []
    f = pd.read_json(path, lines=True)
    for line in f['text']:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            #print('List of tokens:', tokens)
            text_data.append(tokens)

    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]

    pickle.dump(corpus, open('models/corpus.pkl', 'wb'))
    dictionary.save('models/dictionary.gensim')

    NUM_TOPICS = num_topics
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
    ldamodel.save('models/model' + str(NUM_TOPICS) + '.gensim')
    topics = ldamodel.print_topics(num_words=10)
    for topic in topics:
        print(topic)

In [10]:
get_topics('data/qcri/narendramodi_geo.json', 5)

(0, '0.013*"lockdown" + 0.012*"india" + 0.009*"people" + 0.008*"coronavirus" + 0.008*"covid19" + 0.006*"pandemic" + 0.006*"covid-19" + 0.005*"light" + 0.005*"corona" + 0.004*"world"')
(1, '0.017*"corona" + 0.017*"coronavirus" + 0.014*"india" + 0.014*"minister" + 0.010*"prime" + 0.009*"covid19" + 0.008*"fight" + 0.007*"request" + 0.006*"please" + 0.006*"lockdown"')
(2, '0.019*"covid19" + 0.018*"india" + 0.014*"fight" + 0.012*"contribute" + 0.010*"together" + 0.009*"stand" + 0.008*"indiafightscorona" + 0.007*"relief" + 0.007*"care" + 0.007*"corona"')
(3, '0.013*"covid-19" + 0.011*"corona" + 0.010*"indiafightscorona" + 0.009*"lockdown" + 0.009*"covid19" + 0.009*"india" + 0.008*"fight" + 0.008*"indian" + 0.005*"every" + 0.005*"delhi"')
(4, '0.014*"coronavirus" + 0.013*"india" + 0.013*"lockdown" + 0.012*"indian" + 0.010*"covid19" + 0.007*"corona" + 0.007*"covid" + 0.006*"people" + 0.006*"covid-19" + 0.006*"government"')
